# **ÁRBOL BINOMIAL**
# *Eduardo Tomás Leyva Díaz*


## **Modelo Determinista**

Se importan los datos de los precios de CEMEX y también se establecen los valores de precio spot, precio strike, step, volatilidad, tasa libre de riesgo.

Además, se calculan los valores de u, d y p que se utilizarán para hallar el precio de la prima.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

# PRECIOS
datos = "CEMEXCPO.MX"
accion = yf.download(datos, start="2022-03-01", end="2024-03-02")
accion['Rendimientos'] = accion['Close'].pct_change()  # rendimientos discretos

# INPUTS
s0 = accion['Close'].iloc[-1]  # Último Precio Conocido
k = 15
step = 1/12
sigma = accion['Rendimientos'].std() * np.sqrt(252)

r = 0.1025

# Variables del Modelo
u = np.exp(+sigma * np.sqrt(step))
d = np.exp(-sigma * np.sqrt(step))
p = (np.exp(+r * step) - d) / (u - d)
q = 1 - p



[*********************100%***********************]  1 of 1 completed


Luego, se genera un vector con los precios al final del periodo, los cuales van a variar dependiendo el número de veces que subió o bajó.

Adicionalmente, las ganancias potenciales se calculan restando estos precios con el valor **K** que se fijó como precio strike, y esto se realiza para la opción call y la opción put.

In [ ]:
# PRECIOS
periodos = 10
s = [s0 * (u**(periodos-i)) * (d**(i)) for i in range(periodos + 1)]

# GANANCIAS POTENCIALES
gcall = [max(s[i] - k, 0) for i in range(periodos + 1)]
gput = [max(k - s[i], 0) for i in range(periodos + 1)]

print(s)
print(gcall)
print(gput)


[38.54523821892811, 31.179765377399317, 25.2217346139599, 20.40220281446142, 16.50361825042868, 13.350000381469725, 10.798996164408521, 8.735454294127235, 7.066227319932148, 5.715967007064989, 4.62372314766817]
[23.545238218928112, 16.179765377399317, 10.2217346139599, 5.402202814461418, 1.50361825042868, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 1.6499996185302752, 4.201003835591479, 6.264545705872765, 7.933772680067852, 9.28403299293501, 10.37627685233183]


Después, se obtiene el valor esperado de las ganancias potenciales, por lo que se usarán las probabilidades que vienen de una distribución binomial.

Finalmente, solo se trae a valor presente esta esperanza usando el step y la tasa libre de riesgo.

In [ ]:
from scipy.stats import binom

# PROBABILIDADES
# Va de subir 10 veces a subir 0 veces
proba = [binom.pmf(periodos- k, periodos, p) for k in range(periodos + 1)]

# PRIMA CALL
primacall = sum(gcall[i] * proba[i] for i in range(periodos + 1)) * np.exp(-r * step * periodos)
print("Prima Call:", primacall)

# PRIMA PUT
primaput = sum(gput[i] * proba[i] for i in range(periodos + 1)) * np.exp(-r * step * periodos)
print("Prima Put:", primaput)


Prima Call: 1.6388672918048475
Prima Put: 2.0608116703929173


## **Simulación Montecarlo**

Para este caso, primero se creó una función que recibe como argumento el número de periodos y una probabilidad de que el precio suba. Lo que hace es que simula si el precio sube a partir de un número aleatorio y contabiliza las veces que lo hace en ese número de periodos

In [ ]:
# FUNCIÓN
def sube(periodos, proba):
    contador = 0
    for j in range(periodos):
        u = np.random.uniform(0, 1)
        if u < proba:
            contador += 1  # subió
    return contador

Después, en cada simulación se ejecuta la función creada anteriormente y sólo se le asigna la ganancia obtenida dependiendo las veces que sunbió o bajó el precio traida a valor presente. Cada resultado se va guardando en un vector y el valor final de la prima será el promedio de este vector.

**10 simulaciones**

In [ ]:
# PRIMA CALL
n = 10  # simulaciones
primacall = []

for _ in range(n):
    resultado = sube(periodos, p)
    primacall.append(gcall[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Call Promedio:", np.mean(primacall))

# PRIMA PUT
primaput = []

for _ in range(n):
    resultado = sube(periodos, p)
    primaput.append(gput[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Put Promedio:", np.mean(primaput))


Prima Call Promedio: 0.2761032998015804
Prima Put Promedio: 0.3029827146735385


**100 simulaciones**

In [ ]:
# PRIMA CALL
n = 100  # simulaciones
primacall = []

for _ in range(n):
    resultado = sube(periodos, p)
    primacall.append(gcall[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Call Promedio:", np.mean(primacall))

# PRIMA PUT
primaput = []

for _ in range(n):
    resultado = sube(periodos, p)
    primaput.append(gput[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Put Promedio:", np.mean(primaput))


Prima Call Promedio: 1.74805635323589
Prima Put Promedio: 1.8952695254589507


**1,000 simulaciones**

In [ ]:
# PRIMA CALL
n = 1000  # simulaciones
primacall = []

for _ in range(n):
    resultado = sube(periodos, p)
    primacall.append(gcall[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Call Promedio:", np.mean(primacall))

# PRIMA PUT
primaput = []

for _ in range(n):
    resultado = sube(periodos, p)
    primaput.append(gput[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Put Promedio:", np.mean(primaput))


Prima Call Promedio: 1.6137068025706958
Prima Put Promedio: 2.195397679751014


**10,000 simulaciones**

In [ ]:
# PRIMA CALL
n = 10000  # simulaciones
primacall = []

for _ in range(n):
    resultado = sube(periodos, p)
    primacall.append(gcall[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Call Promedio:", np.mean(primacall))

# PRIMA PUT
primaput = []

for _ in range(n):
    resultado = sube(periodos, p)
    primaput.append(gput[10 - resultado] * np.exp(-r * step * periodos))

print("Prima Put Promedio:", np.mean(primaput))


Prima Call Promedio: 1.6245500399174386
Prima Put Promedio: 2.0480504726141064
